In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
data = []
browser = Chrome('E:\CromeDriver\chromedriver.exe') #открытие браузера 
url = 'https://www.kinopoisk.ru/top/navigator/m_act[rating]/1.1%3A/m_act[ex_rating]/1.1%3A/m_act[is_film]/on/m_act[is_mult]/on/order/rating/perpage/200/#results'
browser.get(url)
sleep(5) #ожидание открытия браузера и перехода по ссылке 
x = 5
for p in range(1, 126):

    
    soup = BeautifulSoup(browser.page_source, "lxml") #загрузка кода страницы 
    films = soup.findAll('div', class_='item _NO_HIGHLIGHT_') #поиск всех ячеек с фильмами

    for film in films: #сбор информации из ячейки фильма  
            name = film.find('div', class_='name').find('a').text
            country = film.find('span', class_='gray_text').text.splitlines()[1]
            country = (country + '1')[0:country.find('.') + country.find(',') + 1].strip()
            kp = film.find('div', class_='numVote').text[0:4]
            imdb = film.find('div', class_='imdb').text[6:10]
            genre = film.find('span', class_='gray_text').text.splitlines()[-2].strip()
            genre = genre[1:genre.find('.')].split(', ')

            data.append([name, country, kp, imdb] + genre)

    print(len(data)) #контроль прогресса 
    print(p)
    if p == 125: break
    browser.find_element(By.XPATH ,f'//*[@id="results"]/table/tbody/tr[2]/td/div/div/ul/li[{x}]/a').click() #переход на след. страницу
    if p < 3:     #рассчет позиции кнопки перехода на след. страницу 
        x = p + 7
    else: 
        if 125 - p <= 3:
            x = 125 - p + 6
        else:
            x = 10
    sleep(10)

In [ ]:
pd.DataFrame(data, columns = ['Название', 'Страна', 'КП', 'IMDB', 'Жанр_1', 'Жанр_2', 'Жанр_3'])

In [ ]:
pd.DataFrame(data).to_excel("output.xlsx")